# Complete RAG System with Gemini
End-to-end RAG: Retrieval + Answer Generation

In [ ]:
import sys
sys.path.append('../src')

from retrieval import RetrieverSystem
from llm_generator import AnswerGenerator
from sentence_transformers import SentenceTransformer

## Step 1: Load All Components

In [ ]:
print("Loading components...")

# Load embedding model
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
print("✅ Embedding model loaded")

# Load retriever
retriever = RetrieverSystem.load_index(
    '../index/faiss.index',
    '../index/embeddings.npy',
    '../index/corpus_chunks.json',
    '../index/corpus_meta.json'
)
print("✅ Retriever loaded")

# Load answer generator
generator = AnswerGenerator()
print("✅ Gemini generator loaded")

print("\n🎉 All components ready!")

## Step 2: Complete RAG Pipeline

In [ ]:
def complete_rag(query, k=10):
    """
    Complete RAG pipeline:
    1. Embed query
    2. Retrieve relevant documents
    3. Generate answer with LLM
    """
    # Step 1: Retrieve
    query_emb = model.encode([query])[0]
    contexts = retriever.search(query_emb, k=k)
    
    # Step 2: Generate
    result = generator.generate_answer(query, contexts)
    
    return result

## Step 3: Test with Sample Queries

In [ ]:
# Test queries
test_queries = [
    "كيف أحصل على رخصة ليموزين في قطر؟",
    "ما هي خطوات تسجيل المقررات في جامعة قطر؟",
    "كيف أطلب استشارة طبية عاجلة؟",
]

for query in test_queries:
    print(f"\n{'='*80}")
    print(f"السؤال: {query}")
    print('='*80)
    
    result = complete_rag(query)
    
    print(f"\nالإجابة:\n{result['answer']}")
    
    print(f"\nالمصادر:")
    for i, src in enumerate(result['sources'], 1):
        print(f"  {i}. {src['file']} ({src['category']}) - Score: {src['score']:.3f}")

## Step 4: Interactive Chat

In [ ]:
print("مساعد الخدمات الحكومية - اكتب 'خروج' للإنهاء\n")

chat_history = []

while True:
    query = input("\nسؤالك: ")
    
    if query.strip() in ['خروج', 'exit', 'quit']:
        print("شكراً لاستخدامك المساعد!")
        break
    
    if not query.strip():
        continue
    
    # Get answer
    result = complete_rag(query)
    
    # Display
    print(f"\n{result['answer']}")
    
    # Save to history
    chat_history.append((query, result['answer']))

## Step 5: Evaluate Answer Quality

In [ ]:
# Test specific query in detail
query = "كيف أحصل على رخصة ليموزين؟"

print(f"Query: {query}\n")

# Get retrieval results
query_emb = model.encode([query])[0]
contexts = retriever.search(query_emb, k=10)

print("Top 5 Retrieved Documents:")
for i, ctx in enumerate(contexts[:5], 1):
    print(f"\n{i}. Score: {ctx['score']:.3f}")
    print(f"   Category: {ctx['metadata']['category']}")
    print(f"   File: {ctx['metadata']['source_file'].split('/')[-1]}")
    print(f"   Preview: {ctx['chunk'][:150]}...")

# Generate answer
result = generator.generate_answer(query, contexts)

print(f"\n{'='*80}")
print("Generated Answer:")
print('='*80)
print(result['answer'])

## ✅ Complete RAG System

You now have:
- Semantic search with embeddings
- Fast FAISS retrieval
- Natural language generation with Gemini
- Source attribution
- Arabic language support

Next: Deploy as web app or API!